In [64]:
from sklearn.model_selection import train_test_split
import Read_data_and_Write_results as rw
from collections import defaultdict
from nltk.stem import PorterStemmer as stemmer

In [28]:
import codecs

In [30]:
with codecs.open("train.txt", "r", encoding = "utf-8") as f:
    dataset = f.read()

In [35]:
dataset = dataset.split("\n")

In [37]:
sentences, words = [], []
for word in dataset[1:]:
    if len(word) == 0:
        sentences.append(words)
        words = []
    else:
        words.append(word)

In [40]:
train, test = train_test_split(sentences, test_size=0.2)

In [49]:
def build_dataset(dataset):
    data = ['']
    for words in dataset:
        data.append('\n'.join(words))
        data.append('')
    return '\n'.join(data)

In [54]:
with codecs.open("test.txt", "w", encoding = "utf-8") as f:
    f.write(build_dataset(test))

In [55]:
with codecs.open("semi_train.txt", "w", encoding = "utf-8") as f:
    f.write(build_dataset(train))

In [59]:
_, word_lsts, _, bio_freqs, _, _  = rw.read_data("semi_train.txt")

In [73]:
bio_freqs_word = defaultdict(int)
occurrences_word = defaultdict(int)
for i in range(len(bio_freqs)):
    for j in range(len(bio_freqs[i])):
        bio = bio_freqs[i][j].split("|")
        word_stem = stemmer().stem(word_lsts[i][j].lower())
        bio_freqs_word[word_stem] += (int(bio[0]) + int(bio[1]))
        occurrences_word[word_stem] += 1

In [76]:
def build_model(bio_freqs_word, occurrences):
    model = {}
    for word ,bio_freq in bio_freqs_word.items():
            model[word] = (bio_freq / ( occurrences[word] * 9))
    return model

In [79]:
model = build_model(bio_freqs_word, occurrences_word)

In [80]:
word_id_lsts, word_lsts, _, _, _, _  = rw.read_data("test.txt")

In [82]:
def predict(word_lsts, model):
    predictions = []
    for i in range(len(word_lsts)):
        word_prediction = []
        for j in range(len(word_lsts[i])):
            word_stem = stemmer().stem(word_lsts[i][j].lower())
            if word_stem in model.keys():
                word_prediction.append(model[word_stem])
            else:
                word_prediction.append(0)
        predictions.append(word_prediction)
    return predictions

In [83]:
predictions = predict(word_lsts, model)

In [86]:
rw.write_results(word_id_lsts, word_lsts, predictions, "result.txt")